In [91]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
zip_file_path = '/content/drive/MyDrive/Portfolio 6/3238357.zip'

In [89]:
!unzip -q "$zip_file_path" -d "/content/drive/MyDrive/Portfolio 6"

replace /content/drive/MyDrive/Portfolio 6/images.tar.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [92]:
tar_file_path = '/content/drive/MyDrive/Portfolio 6/bounding_boxes.tar.gz'


In [ ]:
!tar -xvzf "$tar_file_path" -C "/content/drive/MyDrive/Portfolio 6"

In [108]:
tar_file_path = '/content/drive/MyDrive/Portfolio 6/images.tar.gz'

In [109]:
!tar -xvzf "$tar_file_path" -C "/content/drive/MyDrive/Portfolio 6"

images/train/IMG_20180723_123611.jpg
images/train/IMG_20180811_084218.jpg
images/train/IMG_0124.JPG
images/train/IMG_20180723_124259.jpg
images/train/IMG_20180725_115234.jpg
images/train/IMG_20180724_124548.jpg
images/train/IMG_20180719_124416.jpg
images/train/IMG_20180724_121704.jpg
images/test/IMG_1246.jpg
images/test/IMG_20180725_142219.jpg
images/train/IMG_20100101_030428.jpg
images/train/IMG_20180725_122605.jpg
images/train/IMG_20180720_124223.jpg
images/train/IMG_20180721_091154.jpg
images/train/IMG_20180724_120903.jpg
images/train/IMG_20180714_104446.jpg
images/train/IMG_20180714_102503.jpg
images/test/IMG_1263.jpg
images/test/IMG_20100101_033559.jpg
images/train/IMG_20180723_145937.jpg
images/train/IMG_20100101_032407.jpg
images/test/IMG_20100101_033158.jpg
images/train/IMG_1242.jpg
images/train/IMG_20180723_152110.jpg
images/train/IMG_20180811_084135.jpg
images/test/IMG_0234.JPG
images/train/IMG_20180717_120936.jpg
images/train/IMG_20100101_033626.jpg
images/train/IMG_20180721

In [101]:
# Define directories
train_annotation_csv = '/content/drive/MyDrive/Portfolio 6/Bounding_boxes/train_labels.csv'
test_annotation_csv = '/content/drive/MyDrive/Portfolio 6/Bounding_boxes/test_labels.csv'
train_image_dir = '/content/drive/MyDrive/Portfolio 6/images/train'
test_image_dir = '/content/drive/MyDrive/Portfolio 6/images/test'
train_label_dir = '/content/drive/MyDrive/Portfolio 6/labels/train'
test_label_dir = '/content/drive/MyDrive/Portfolio 6/labels/test'
# Target directories for YOLO formatted data
train_target_image_dir = '/content/drive/MyDrive/Portfolio 6/yolo_data/train/images'
train_target_label_dir = '/content/drive/MyDrive/Portfolio 6/yolo_data/train/labels'
test_target_image_dir = '/content/drive/MyDrive/Portfolio 6/yolo_data/test/images'
test_target_label_dir = '/content/drive/MyDrive/Portfolio 6/yolo_data/test/labels'

In [116]:
import os
import shutil
import pandas as pd

def convert_to_yolo_format(image_width, image_height, x_min, y_min, x_max, y_max):
    center_x = (x_min + x_max) / 2 / image_width
    center_y = (y_min + y_max) / 2 / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    return center_x, center_y, width, height

def generate_yolo_labels(annotation_csv, output_dir, image_dir):
    os.makedirs(output_dir, exist_ok=True)
    annotation_df = pd.read_csv(annotation_csv)

    for _, row in annotation_df.iterrows():
        image_name = row['filename']
        image_path = os.path.join(image_dir, image_name)

        if not os.path.exists(image_path):
            print(f"Warning: Image {image_name} not found. Skipping this annotation.")
            continue

        image_width, image_height = row['width'], row['height']
        x_min, y_min, x_max, y_max = row['xmin'], row['ymin'], row['xmax'], row['ymax']

        # Convert to YOLO format
        center_x, center_y, width, height = convert_to_yolo_format(image_width, image_height, x_min, y_min, x_max, y_max)

        # Write the YOLO label to a .txt file
        label_filename = os.path.splitext(image_name)[0] + '.txt'
        label_filepath = os.path.join(output_dir, label_filename)

        with open(label_filepath, 'w') as label_file:
            label_file.write(f"{row['class']} {center_x} {center_y} {width} {height}\n")

# Function to copy selected images and their corresponding YOLO labels to the target directory
def prepare_yolo_data(image_dir, label_dir, target_image_dir, target_label_dir, selected_images):
    os.makedirs(target_image_dir, exist_ok=True)
    os.makedirs(target_label_dir, exist_ok=True)

    for image in selected_images:
        image_path = os.path.join(image_dir, image)
        label_name = os.path.splitext(image)[0] + '.txt'
        label_path = os.path.join(label_dir, label_name)
        target_image_path = os.path.join(target_image_dir, image)
        target_label_path = os.path.join(target_label_dir, label_name)

        # Check if the source and destination are not the same file
        if image_path != target_image_path:
            shutil.copy(image_path, target_image_path)

        if label_path != target_label_path:
            if os.path.exists(label_path):
                shutil.copy(label_path, target_label_path)
            else:
                print(f"Warning: Label file not found for {image}, skipping this image.")

def select_random_train_images(image_dir, num_samples=400):
    all_train_images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    num_images = len(all_train_images)

    if num_images < num_samples:
        print(f"Warning: Only {num_images} images available for training. Adjusting to select all.")
        num_samples = num_images  # Adjust to the available number of images

    return random.sample(all_train_images, num_samples)

def select_random_test_images(image_dir, num_samples=40):
    all_test_images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    num_images = len(all_test_images)

    if num_images < num_samples:
        print(f"Warning: Only {num_images} images available for testing. Adjusting to select all.")
        num_samples = num_images  # Adjust to the available number of images

    return random.sample(all_test_images, num_samples)

In [117]:
# Generate YOLO labels for all training images
generate_yolo_labels(train_annotation_csv, train_target_label_dir, train_image_dir)

# Randomly select images for training
random_train_images = select_random_train_images(train_image_dir, num_samples=400)

# Prepare YOLO formatted training data
prepare_yolo_data(train_image_dir, train_label_dir, train_target_image_dir, train_target_label_dir, random_train_images)


In [75]:
# Randomly select 40 images for testing
random_test_images = select_random_test_images(test_image_dir, num_samples=40)

# Generate YOLO labels for the randomly selected test images
generate_yolo_labels(test_annotation_csv, test_target_label_dir, test_image_dir, random_test_images)

# Prepare YOLO formatted test data
prepare_yolo_data(test_image_dir, test_target_label_dir, test_target_image_dir, test_target_label_dir, random_test_images)

**Train the YOLO Model**

In [34]:
!pip install -U torch torchvision
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

Cloning into 'yolov5'...
remote: Enumerating objects: 17018, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 17018 (delta 120), reused 124 (delta 68), pack-reused 16805 (from 1)
Receiving objects: 100% (17018/17018), 15.73 MiB | 22.88 MiB/s, done.
Resolving deltas: 100% (11658/11658), done.
/content/yolov5/yolov5/yolov5


In [95]:
yaml_content = """
train: /content/drive/MyDrive/Portfolio 6/yolo_data/train
val: /content/drive/MyDrive/Portfolio 6/images/train

nc: 1
names: ['graffiti']
"""

with open('/content/drive/MyDrive/Portfolio 6/yolo_data/yolov5_data.yaml', 'w') as file:
    file.write(yaml_content)



In [77]:
%cd /content/yolov5

/content/yolov5


**Change Grafitti to 0**

In [98]:
import os

# Define the directory where your label files are located
label_dir = '/content/drive/MyDrive/Portfolio 6/yolo_data/train/labels'  # Update this path as needed
class_id = 0  # Update this to the appropriate class ID for "Graffiti"

# Iterate through each label file in the specified directory
for filename in os.listdir(label_dir):
    if filename.endswith('.txt'):
        # Full path to the label file
        file_path = os.path.join(label_dir, filename)

        # Read the content of the label file
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Open the file again to write updated content
        with open(file_path, 'w') as file:
            for line in lines:
                # Replace "Graffiti" with the class ID if it appears
                if 'Graffiti' in line:
                    # Split the line into parts
                    parts = line.strip().split()
                    # Update the class ID to the numeric class ID
                    parts[0] = str(class_id)  # Update class ID
                    # Write the modified line back to the file
                    file.write(' '.join(parts) + '\n')
                else:
                    # Write the line back unchanged if it doesn't contain "Graffiti"
                    file.write(line)

print("Replacement complete.")

Replacement complete.


In [100]:
!python train.py --img 640 --batch 16 --epochs 50 --data "/content/drive/MyDrive/Portfolio 6/yolo_data/yolov5_data.yaml" --weights yolov5s.pt --cache


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-10-21 15:41:22.131261: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 15:41:22.156685: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 15:41:22.164948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: thinhthomas2000 (thinhthomas2000-swinburne-university-of-techno